In [1]:
import pandas as pd
import joblib

# --------- Load Saved Models ---------
classifier = joblib.load("/home/abdeldjalil-hani/Desktop/emergency-sorting-system/saved-models/classifier_voting_model.pkl")
regressors = {
    i: joblib.load(f"/home/abdeldjalil-hani/Desktop/emergency-sorting-system/saved-models/regressor_level_{i}.pkl")
    for i in ['Resuscitation (L1)', 'Emergent (L2)', 'Urgent (L3)', 'Less Urgent (L4)', 'Non-Urgent (L5)']
}

In [2]:
encodings = {
    'Residence_type': {'Rural': 0, 'Urban': 1},
    'smoking_status': {
        'Unknown': 0, 'formerly smoked': 1, 'never smoked': 2, 'smokes': 3
    },
    'Symptom': {
        'Abdominal pain': 0, 'Abdominal pain, Chest pain': 1, 'Abdominal pain, Difficulty breathing': 2,
        'Abdominal pain, Fever': 3, 'Abdominal pain, Headache': 4, 'Abdominal pain, Weakness': 5,
        'Chest pain': 6, 'Chest pain, Abdominal pain': 7, 'Chest pain, Difficulty breathing': 8,
        'Chest pain, Fever': 9, 'Chest pain, Headache': 10, 'Chest pain, Weakness': 11,
        'Difficulty breathing': 12, 'Difficulty breathing, Abdominal pain': 13, 'Difficulty breathing, Chest pain': 14,
        'Difficulty breathing, Fever': 15, 'Difficulty breathing, Headache': 16, 'Difficulty breathing, Weakness': 17,
        'Fever': 18, 'Fever, Abdominal pain': 19, 'Fever, Chest pain': 20, 'Fever, Difficulty breathing': 21,
        'Fever, Headache': 22, 'Fever, Weakness': 23, 'Headache': 24, 'Headache, Abdominal pain': 25,
        'Headache, Chest pain': 26, 'Headache, Difficulty breathing': 27, 'Headache, Fever': 28,
        'Headache, Weakness': 29, 'Weakness': 30, 'Weakness, Abdominal pain': 31,
        'Weakness, Chest pain': 32, 'Weakness, Difficulty breathing': 33,
        'Weakness, Fever': 34, 'Weakness, Headache': 35
    },
    'Consciousness': {'Awake': 0, 'Responds to Pain': 1, 'Unconscious': 2},
    'Risk Factors': {
        'Cancer': 0, 'Cancer, Cardiovascular disease': 1, 'Cancer, Diabetes': 2,
        'Cancer, Hypertension': 3, 'Cancer, None': 4, 'Cancer, kidney failure': 5,
        'Cardiovascular disease': 6, 'Cardiovascular disease, Cancer': 7, 'Cardiovascular disease, Diabetes': 8,
        'Cardiovascular disease, Hypertension': 9, 'Cardiovascular disease, None': 10,
        'Cardiovascular disease, kidney failure': 11, 'Diabetes': 12, 'Diabetes, Cancer': 13,
        'Diabetes, Cardiovascular disease': 14, 'Diabetes, Hypertension': 15, 'Diabetes, None': 16,
        'Diabetes, kidney failure': 17, 'Hypertension': 18, 'Hypertension, Cancer': 19,
        'Hypertension, Cardiovascular disease': 20, 'Hypertension, Diabetes': 21, 'Hypertension, None': 22,
        'Hypertension, kidney failure': 23, 'kidney failure': 24, 'kidney failure, Cancer': 25,
        'kidney failure, Cardiovascular disease': 26, 'kidney failure, Diabetes': 27,
        'kidney failure, Hypertension': 28, 'kidney failure, None': 29, 'none risk factor': 30
    },
    'Massive Bleeding': {False: 0, True: 1},
    'Respiratory Distress': {False: 0, True: 1}
}

In [3]:
# --------- Example Patients (Preprocessed) ---------
patients = [
    # Cluster 0
    {
        'age': 61, 'gender': 0.0, 'chest pain type': 0, 'cholesterol': 152, 'exercise angina': 0,
        'plasma glucose': 106.58, 'skin_thickness': 35, 'bmi': 23.9, 'hypertension': 0, 'heart_disease': 0,
        'Residence_type': 'Rural', 'smoking_status': 'Unknown', 'Symptom': 'Difficulty breathing',
        'Temperature (¬∞C)': 36.4, 'Heart Rate (bpm)': 90, 'Respiratory Rate (breaths/min)': 19,
        'SpO2 (%)': 96, 'Glasgow Score': 13, 'Consciousness': 'Awake',
        'Massive Bleeding': False, 'Respiratory Distress': True, 'Risk Factors': 'kidney failure',
        'Blood Pressure (mmHg)': '115/82'
    },
    {
        'age': 58, 'gender': 0.0, 'chest pain type': 0, 'cholesterol': 177, 'exercise angina': 0,
        'plasma glucose': 149.75, 'skin_thickness': 76, 'bmi': 27.0, 'hypertension': 0, 'heart_disease': 0,
        'Residence_type': 'Urban', 'smoking_status': 'Unknown', 'Symptom': 'Abdominal pain',
        'Temperature (¬∞C)': 36.5, 'Heart Rate (bpm)': 61, 'Respiratory Rate (breaths/min)': 18,
        'SpO2 (%)': 99, 'Glasgow Score': 15, 'Consciousness': 'Awake',
        'Massive Bleeding': False, 'Respiratory Distress': False, 'Risk Factors': 'Cardiovascular disease',
        'Blood Pressure (mmHg)': '121/78'
    },

    # Cluster 1
    {
        'age': 54, 'gender': 0.0, 'chest pain type': 2, 'cholesterol': 294, 'exercise angina': 0,
        'plasma glucose': 119.0, 'skin_thickness': 24, 'bmi': 23.0, 'hypertension': 0, 'heart_disease': 0,
        'Residence_type': 'Urban', 'smoking_status': 'never smoked', 'Symptom': 'Headache',
        'Temperature (¬∞C)': 37.1, 'Heart Rate (bpm)': 71, 'Respiratory Rate (breaths/min)': 17,
        'SpO2 (%)': 99, 'Glasgow Score': 15, 'Consciousness': 'Awake',
        'Massive Bleeding': False, 'Respiratory Distress': False, 'Risk Factors': 'kidney failure',
        'Blood Pressure (mmHg)': '121/79'
    },
    {
        'age': 45, 'gender': 0.0, 'chest pain type': 2, 'cholesterol': 294, 'exercise angina': 0,
        'plasma glucose': 113.0, 'skin_thickness': 21, 'bmi': 18.0, 'hypertension': 0, 'heart_disease': 0,
        'Residence_type': 'Urban', 'smoking_status': 'never smoked', 'Symptom': 'Headache, Chest pain',
        'Temperature (¬∞C)': 36.5, 'Heart Rate (bpm)': 140, 'Respiratory Rate (breaths/min)': 21,
        'SpO2 (%)': 91, 'Glasgow Score': 12, 'Consciousness': 'Responds to Pain',
        'Massive Bleeding': False, 'Respiratory Distress': True, 'Risk Factors': 'Hypertension',
        'Blood Pressure (mmHg)': '137/72'
    },

    # Cluster 2
    {
        'age': 54, 'gender': 0.0, 'chest pain type': 0, 'cholesterol': 198, 'exercise angina': 0,
        'plasma glucose': 180.93, 'skin_thickness': 65, 'bmi': 27.7, 'hypertension': 1, 'heart_disease': 0,
        'Residence_type': 'Urban', 'smoking_status': 'never smoked', 'Symptom': 'Difficulty breathing',
        'Temperature (¬∞C)': 36.2, 'Heart Rate (bpm)': 92, 'Respiratory Rate (breaths/min)': 19,
        'SpO2 (%)': 98, 'Glasgow Score': 14, 'Consciousness': 'Awake',
        'Massive Bleeding': False, 'Respiratory Distress': True, 'Risk Factors': 'none risk factor',
        'Blood Pressure (mmHg)': '112/76'
    },
    {
        'age': 60, 'gender': 0.0, 'chest pain type': 0, 'cholesterol': 199, 'exercise angina': 0,
        'plasma glucose': 97.43, 'skin_thickness': 41, 'bmi': 26.4, 'hypertension': 0, 'heart_disease': 0,
        'Residence_type': 'Urban', 'smoking_status': 'smokes', 'Symptom': 'Chest pain',
        'Temperature (¬∞C)': 37.7, 'Heart Rate (bpm)': 76, 'Respiratory Rate (breaths/min)': 19,
        'SpO2 (%)': 92, 'Glasgow Score': 15, 'Consciousness': 'Awake',
        'Massive Bleeding': False, 'Respiratory Distress': False, 'Risk Factors': 'Cardiovascular disease',
        'Blood Pressure (mmHg)': '138/90'
    },

    # Cluster 3
    {
        'age': 49, 'gender': 0.0, 'chest pain type': 0, 'cholesterol': 188, 'exercise angina': 0,
        'plasma glucose': 138.16, 'skin_thickness': 59, 'bmi': 19.4, 'hypertension': 0, 'heart_disease': 0,
        'Residence_type': 'Rural', 'smoking_status': 'never smoked', 'Symptom': 'Abdominal pain, Weakness',
        'Temperature (¬∞C)': 36.9, 'Heart Rate (bpm)': 77, 'Respiratory Rate (breaths/min)': 13,
        'SpO2 (%)': 97, 'Glasgow Score': 13, 'Consciousness': 'Awake',
        'Massive Bleeding': False, 'Respiratory Distress': False, 'Risk Factors': 'Cardiovascular disease',
        'Blood Pressure (mmHg)': '110/88'
    },
    {
        'age': 50, 'gender': 0.0, 'chest pain type': 2, 'cholesterol': 202, 'exercise angina': 0,
        'plasma glucose': 89.0, 'skin_thickness': 57, 'bmi': 19.0, 'hypertension': 0, 'heart_disease': 0,
        'Residence_type': 'Urban', 'smoking_status': 'never smoked', 'Symptom': 'Headache, Abdominal pain',
        'Temperature (¬∞C)': 37.9, 'Heart Rate (bpm)': 70, 'Respiratory Rate (breaths/min)': 17,
        'SpO2 (%)': 100, 'Glasgow Score': 13, 'Consciousness': 'Awake',
        'Massive Bleeding': False, 'Respiratory Distress': False, 'Risk Factors': 'Diabetes',
        'Blood Pressure (mmHg)': '126/80'
    },

    # Cluster 4
    {
        'age': 79, 'gender': 0.0, 'chest pain type': 0, 'cholesterol': 154, 'exercise angina': 0,
        'plasma glucose': 110.85, 'skin_thickness': 96, 'bmi': 24.1, 'hypertension': 0, 'heart_disease': 0,
        'Residence_type': 'Urban', 'smoking_status': 'formerly smoked', 'Symptom': 'Chest pain',
        'Temperature (¬∞C)': 37.4, 'Heart Rate (bpm)': 100, 'Respiratory Rate (breaths/min)': 14,
        'SpO2 (%)': 100, 'Glasgow Score': 13, 'Consciousness': 'Awake',
        'Massive Bleeding': False, 'Respiratory Distress': False, 'Risk Factors': 'Hypertension',
        'Blood Pressure (mmHg)': '121/79'
    },
    {
        'age': 56, 'gender': 0.0, 'chest pain type': 0, 'cholesterol': 196, 'exercise angina': 0,
        'plasma glucose': 185.17, 'skin_thickness': 85, 'bmi': 40.4, 'hypertension': 0, 'heart_disease': 0,
        'Residence_type': 'Urban', 'smoking_status': 'formerly smoked', 'Symptom': 'Chest pain',
        'Temperature (¬∞C)': 37.3, 'Heart Rate (bpm)': 74, 'Respiratory Rate (breaths/min)': 18,
        'SpO2 (%)': 95, 'Glasgow Score': 15, 'Consciousness': 'Awake',
        'Massive Bleeding': False, 'Respiratory Distress': False, 'Risk Factors': 'Hypertension',
        'Blood Pressure (mmHg)': '118/79'
    }
]

In [4]:
# --------- Process & Predict ---------
def preprocess_and_predict(patient_dict):
    patient = patient_dict.copy()
    
    # Encode categorical fields
    for col, mapping in encodings.items():
        if col in patient:
            patient[col] = mapping[patient[col]]

    # Split blood pressure into systolic & diastolic
    if 'Blood Pressure (mmHg)' in patient:
        bp = patient.pop('Blood Pressure (mmHg)').split('/')
        patient['blood_pressure'] = int(bp[0])
        patient['heart_pressure'] = int(bp[1])

    # Ensure all feature columns are in correct order
    feature_order = [
        'age', 'gender', 'chest pain type', 'cholesterol', 'exercise angina',
        'plasma glucose', 'skin_thickness', 'bmi', 'hypertension', 'heart_disease',
        'Residence_type', 'smoking_status', 'Symptom', 'Temperature (¬∞C)',
        'Heart Rate (bpm)', 'Respiratory Rate (breaths/min)', 'SpO2 (%)', 'Glasgow Score',
        'Consciousness', 'Massive Bleeding', 'Respiratory Distress',
        'Risk Factors', 'blood_pressure', 'heart_pressure'
    ]

    patient_df = pd.DataFrame([patient])[feature_order]
    cls = classifier.predict(patient_df)[0]
    score = regressors[cls].predict(patient_df)[0]

    return cls, score

In [5]:
# --------- Run Predictions ---------
for i, p in enumerate(patients):
    cls, score = preprocess_and_predict(p)
    print(f"Patient {i+1}: Emergency Level = {cls}, Score = {score:.2f}")

Patient 1: Emergency Level = Less Urgent (L4), Score = 94.84
Patient 2: Emergency Level = Less Urgent (L4), Score = 87.21
Patient 3: Emergency Level = Emergent (L2), Score = 97.07
Patient 4: Emergency Level = Non-Urgent (L5), Score = 94.32
Patient 5: Emergency Level = Less Urgent (L4), Score = 101.30
Patient 6: Emergency Level = Emergent (L2), Score = 96.47
Patient 7: Emergency Level = Urgent (L3), Score = 92.72
Patient 8: Emergency Level = Emergent (L2), Score = 97.07
Patient 9: Emergency Level = Less Urgent (L4), Score = 94.26
Patient 10: Emergency Level = Less Urgent (L4), Score = 94.26
